In [6]:
import pandas as pd
import json
import hashlib
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)


# Genero variables con las rutas de los archivos y los coloco en una lista

json1 = r"datasets\originals\google_places\1.json"
json2 = r"datasets\originals\google_places\2.json"
json3 = r"datasets\originals\google_places\3.json"
json4 = r"datasets\originals\google_places\4.json"
json5 = r"datasets\originals\google_places\5.json"
json6 = r"datasets\originals\google_places\6.json"
json7 = r"datasets\originals\google_places\7.json"
json8 = r"datasets\originals\google_places\8.json"
json9 = r"datasets\originals\google_places\9.json"
json10 = r"datasets\originals\google_places\10.json"
json11 = r"datasets\originals\google_places\11.json"

In [7]:
json_list = [json1, json2, json3, json4, json5, json6, json7, json8, json9, json10, json11]

marriot_hotels = pd.read_csv(r"datasets\Marriott_Hotels.csv", encoding='latin1', sep=";")

id_category = categories_df = pd.read_csv("datasets\dataw\categorias.csv", encoding='latin1', sep=",")

In [8]:
# Esta funcion convierte el archivo json de google metadata_sitios en un dataframe

def json_to_df(json_file):
    data = []
    with open(json_file, 'r') as f:
        for line in f:
            record = json.loads(line)
            data.append(record)
    
    df = pd.DataFrame(data)
    return df

In [9]:
# esta funcion separa address en direccion, estado, ciudad y CP. Tambien elimina columnas innecesarias

def split_address_column(df):

    address_split = df['address'].str.split(', ', expand=True)
    df['nombre'] = address_split[0]
    df['direccion'] = address_split[1]
    df['ciudad'] = address_split[2]
    df['estado'] = address_split[3]
    estado_codigo_postal = address_split[3].str.split(' ', expand=True)
    df['estado'] = estado_codigo_postal[0]
    df['codigo_postal'] = estado_codigo_postal[1]
    df.drop(columns=['address'], inplace=True)
    df.drop(columns=['name'], inplace=True)
    df.drop(columns=['description'], inplace=True)
    df.drop(columns=['price'], inplace=True)
    df.drop(columns=['relative_results'], inplace=True)
    df.drop(columns=['url'], inplace=True)
    df.drop(columns=['MISC'], inplace=True)
        
    return df

In [10]:
# esta funcion filtra los dataframe y obtiene solamente los estados de Florida, Nevada y California

def filter_states(df):
    
    df['estado'] = df['estado'].str.strip()
    df['estado'] = df['estado'].str.upper()
    df['estado'] = df['estado'].str.replace('FLORIDA', 'FL')
    df['estado'] = df['estado'].str.replace('CALIFORNIA', 'CA')
    df['estado'] = df['estado'].str.replace('NEVADA', 'NV')
    df = df[df['estado'].isin(['FL', 'NV', 'CA'])]
    df.reset_index(drop=True, inplace=True)

    return df

In [11]:
# Esta funcion obtiene los locales de las ciudades de Miami Los Angeles Las Vegas Paradise Winchester

def filter_cities(df):

    target_cities = ["Miami", "Los Angeles", "Las Vegas", "Paradise", "Winchester"]
    df = df[df["ciudad"].isin(target_cities)]
    
    return df

In [12]:
# Esta funcion elimina los comercios que esten cerrados permanentemente o temporalmente

def remove_closed_shops(df):

    df = df[(df["state"] != "Permanently closed") & (df["state"] != "Temporarily closed")]
    
    return df

In [13]:
# Esta funcion quita los valores donde en la colulmna categoria sean nulos

def remove_null_category(df):

    df = df[df['category'].notnull()]
    
    return df

In [14]:
# Esta funcion filtra los shops de google por los locales gastronomicos

def filter_by_keywords(data, keywords):
    
    patron = '|'.join(keywords)
    filtro = data['category'].str.contains(patron, case=False)
    filas_filtradas = data[filtro]
    return filas_filtradas

filter_gastronomic = ['Grill', 'Pub', 'Deli', 'Mexican', 'Ice Cream', 'Brewery', 'Sushi', 'Restaurants', 'Restaurant', 'Hoteles', 'Hotel', 'Restaurante', 'Bar', 'Cafe', 'Coffe', 'Pizza', 'Bakery', 'Food', 'Diner', 'Bistro', 'Bakeries']

In [15]:
# Esta funcion elimina los duplicados en el dataframe

def remove_duplicates(df):

    df = df.drop_duplicates(subset=['gmap_id'])
    return df

In [16]:
# Esta funcion genera un ID hash con la suma de la latitud y la longitud, luego chequea que no haya duplicados.
# En caso de haber duplicados, elimina el duplicado de menor valor en la columna 'avg_rating'

def generate_hash(latitude, longitude):
    hash_input = str(float(latitude) + float(longitude))
    hash_object = hashlib.md5(hash_input.encode())
    return hash_object.hexdigest()

def check_duplicates_hash(df, id_column, avg_rating_column):
    
    df_ordenado = df.sort_values(by=avg_rating_column)
    df_sin_duplicados = df_ordenado.drop_duplicates(subset=id_column, keep='first')
    
    return df_sin_duplicados


In [17]:
# Esta funcion genera la tabla de horarios en un nuevo dataframe

def generate_new_dataframe_with_hours(df):
    
    new_rows = []
    for index, row in df.iterrows():
        id_hash = row['id_hash']
        hours = row['hours']
        if hours:
            for day_info in hours:
                day_name = day_info[0]
                times = day_info[1].split('–')
                open_time = times[0]
                close_time = times[1] if len(times) > 1 else None
                new_rows.append({
                    'id_hash': id_hash,
                    'day_name': day_name,
                    'open_time': open_time,
                    'close_time': close_time
                })
    new_df = pd.DataFrame(new_rows)
    df.drop('hours', axis=1, inplace=True)
    new_df = new_df.rename(columns={'id_hash': 'shop_id'})
    column_order = ['day_name', 'open_time', 'close_time', 'shop_id']
    new_df = new_df.reindex(columns=column_order)

    return new_df

In [18]:
# Esta funcion convierte las columnas de horarios en formato 24hs

def convert_to_24hr(time_str):

    if pd.isnull(time_str):
        return np.nan
    time_str = time_str.replace('AM', '').replace('PM', '')
    if ':' in time_str:
        hours, minutes = map(int, time_str.split(':'))
        if 'PM' in time_str and hours != 12:
            hours += 12
    elif time_str.strip().lower() == 'closed':
        return np.nan
    else:
        try:
            hours = int(time_str)
            minutes = 0
        except ValueError:
            return np.nan
            
    return f"{hours:02d}:{minutes:02d}"

In [2]:
# Esta funcion genera la dimension location para el Dw y corrije nombres y posiciones de columna 

def generate_location_dataframe(df):
    
    location_df = df[['id_hash', 'estado', 'latitude', 'longitude', 'codigo_postal', 'ciudad', 'direccion']].copy()
    location_df.columns = ['id_shop', 'state', 'latitude', 'longitude', 'postal_code', 'city', 'address']
    df.drop(['latitude', 'longitude', 'estado', 'codigo_postal', 'ciudad', 'direccion'], axis=1, inplace=True)
    location_df['postal_code'] = location_df['postal_code'].str.replace(r'\D', '', regex=True)
    location_df['postal_code'] = np.where(location_df['postal_code'] != '', location_df['postal_code'], None)
    location_df['postal_code'] = location_df['postal_code'].astype('Int64')
    location_df['id_shop'] = location_df['id_shop'].astype(str)
    
    return location_df

In [20]:
# Esta funcion genera una nueva dimension de categorias para el dw

def extract_categories(df):
    
    new_df = df[['category', 'id_hash']].copy()
    new_df['category'] = new_df['category'].apply(lambda x: x.replace(',', '').split())
    new_df = new_df.explode('category')
    new_df.reset_index(drop=True, inplace=True)

    return new_df

In [21]:
# Esta funcion normaliza la dimension category_shop, cambiando la categoria por el id de la categoria.
# luego se queda solo con las categorias con ID

def process_category_shop(df, id_category):

    df = df.sort_values(by='id_hash')
    df['category'] = df['category'].str.title()
    id_category['category'] = id_category['category'].str.title()
    df = df.merge(id_category, on='category', how='left')
    df = df.dropna(subset=['id_category'])
    df['id_category'] = df['id_category'].astype('int64')
    df = df.drop(columns=['category']).rename(columns={'id_hash': 'id_shop'})
    df = df.reset_index(drop=True)
    
    return df

In [22]:
# Esta funcion prepara el dataframe shops para el DW

def transform_shops(df):

    df['source'] = 2
    df = df.rename(columns={
        'id_hash': 'id_shop',
        'nombre': 'name',
        'avg_rating': 'stars',
        'gmap_id': 'id_source'
    })
    column_order = ['id_shop', 'name', 'stars', 'source', 'id_source']
    df = df[column_order]
    
    return df


In [23]:
# esta funcion ejecula las func para que desde los archivos json podamos obtener un dataframe
# con la columna adress, discriminada en varias columnas
# y filtrada por las ciudades que nos interesan

def process_and_concat(json_list):
    
    dfs = []
    for json_file in json_list:
        df = json_to_df(json_file)
        df = split_address_column(df)
        df = filter_cities(df)
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df

In [24]:
# Esta funcion procesa las funciones de normalizacion

def normalization_functions(df):
    
    df = remove_duplicates(df)
    df = remove_null_category(df)
    df = remove_closed_shops(df)
    df['category'] = df['category'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    df = filter_by_keywords(df, filter_gastronomic)
    df['id_hash'] = df.apply(lambda row: generate_hash(row['latitude'], row['longitude']), axis=1)
    df = check_duplicates_hash(df, id_column='id_hash', avg_rating_column='avg_rating')
    df = df.reset_index(drop=True)

    return df

In [25]:
# Genero el dataframe inicial

google_shops = process_and_concat(json_list) 

In [65]:
# genero df solo con los locales gastronomicos y hoteleros

google_shops_gastronomy = normalization_functions(google_shops)

In [66]:
# Genero la nueva dimension del DW day_time

day_time_g = generate_new_dataframe_with_hours(google_shops_gastronomy)
day_time_g['open_time'] = day_time_g['open_time'].apply(convert_to_24hr)
day_time_g['close_time'] = day_time_g['close_time'].apply(convert_to_24hr)

day_time_g.to_csv(r"datasets\dataw\day_time_g.csv", sep=';', index=False)
day_time_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34244 entries, 0 to 34243
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   day_name    34244 non-null  object
 1   open_time   29269 non-null  object
 2   close_time  29269 non-null  object
 3   shop_id     34244 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB


In [67]:
day_time_g.head()

,day_name,open_time,close_time,shop_id
0,Friday,09:00,05:00,d4cbb312e58f20251b83cf3157d37b83
1,Saturday,09:00,05:00,d4cbb312e58f20251b83cf3157d37b83
2,Sunday,09:00,05:00,d4cbb312e58f20251b83cf3157d37b83
3,Monday,09:00,05:00,d4cbb312e58f20251b83cf3157d37b83
4,Tuesday,09:00,05:00,d4cbb312e58f20251b83cf3157d37b83


In [68]:
# Genero la nueva dimension de DW location

location_g = generate_location_dataframe(google_shops_gastronomy)
location_g.to_csv(r"datasets\dataw\location_g.csv", sep=';', index=False)
location_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5803 entries, 0 to 5802
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_shop      5803 non-null   object 
 1   state        5803 non-null   object 
 2   latitude     5803 non-null   float64
 3   longitude    5803 non-null   float64
 4   postal_code  5800 non-null   Int64  
 5   city         5803 non-null   object 
 6   adress       5803 non-null   object 
dtypes: Int64(1), float64(2), object(4)
memory usage: 323.1+ KB


In [69]:
location_g.head()

,id_shop,state,latitude,longitude,postal_code,city,adress
0,d86aef7d13e277917b5323e437352cd3,FL,25.777883,-80.406450,33182,Miami,12976 NW 9th St
1,769f91eacf53036927de048fb4874d25,FL,25.775277,-80.188582,33132,Miami,19100 NE 3rd Ave
2,90f3c5b2f79b3eba1be81e46518dd5fe,NV,36.147183,-115.187865,89102,Las Vegas,2216 Plaza Del Prado
3,f1b3b66602977f1c653f7cb0c5d6b6cf,NV,36.165312,-115.153391,89155,Las Vegas,500 S Grand Central Pkwy
4,d4cbb312e58f20251b83cf3157d37b83,NV,36.195477,-115.248844,89128,Las Vegas,7175 W Lake Mead Blvd


In [70]:
# Genero la nueva dimension de DW category_shop

category_shop_g = extract_categories(google_shops_gastronomy)
category_shop_g = process_category_shop(category_shop_g, id_category)
category_shop_g.to_csv(r"datasets\dataw\category_shop_g.csv", sep=';', index=False)
category_shop_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9485 entries, 0 to 9484
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_shop      9485 non-null   object
 1   id_category  9485 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 148.3+ KB


In [71]:
category_shop_g.head()

,id_shop,id_category
0,001d1c3dc79b1ea635c4bacbd9f75c96,23
1,001d1c3dc79b1ea635c4bacbd9f75c96,20
2,001d1c3dc79b1ea635c4bacbd9f75c96,20
3,001d1c3dc79b1ea635c4bacbd9f75c96,20
4,001d1c3dc79b1ea635c4bacbd9f75c96,23


In [72]:
# Genero la nueva dimension de DW shops

shops_g = transform_shops(google_shops_gastronomy)
shops_g.to_csv(r"datasets\dataw\shops_g.csv", sep=';', index=False)
shops_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5803 entries, 0 to 5802
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_shop    5803 non-null   object 
 1   name       5803 non-null   object 
 2   stars      5803 non-null   float64
 3   source     5803 non-null   int64  
 4   id_source  5803 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 226.8+ KB


In [73]:
shops_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5803 entries, 0 to 5802
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_shop    5803 non-null   object 
 1   name       5803 non-null   object 
 2   stars      5803 non-null   float64
 3   source     5803 non-null   int64  
 4   id_source  5803 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 226.8+ KB
